In [1]:
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim_models

import pandas as pd
import os
import numpy as np

In [2]:
parent_dir = os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
folder_path = r'Data\PreprocessedData'
net_path = os.path.join(parent_dir, folder_path)

In [3]:
df_train_english = pd.read_csv(os.path.join(net_path, 'english_train_preprocess.csv'))
df_dev_english = pd.read_csv(os.path.join(net_path, 'english_dev_preprocess.csv'))
df_test_english = pd.read_csv(os.path.join(net_path, 'english_test_preprocess.csv'))

In [4]:
train_english_sentences = df_train_english['preprocessed_text'].tolist()
dev_english_sentences = df_dev_english['preprocessed_text'].tolist()
test_english_sentences = df_test_english['preprocessed_text'].tolist()

In [5]:
english_sentences = train_english_sentences + dev_english_sentences + test_english_sentences

In [6]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)

In [7]:
lemmatized_texts = lemmatization(english_sentences)

In [9]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print(data_words[0][0:20])

['tiktok', 'radiate', 'gay', 'chaotic', 'energy', 'love']


In [10]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]
chaotic


In [11]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

## Visualize

In [13]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

C:\Users\ASUS\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10     0.529921 -0.124619       1        1  14.869712
24     0.491232  0.139009       2        1   7.725244
8     -0.487079  0.150958       3        1   6.780175
26    -0.468411 -0.151164       4        1   5.775153
19     0.051104 -0.491650       5        1   4.988085
12    -0.350411 -0.277453       6        1   3.853072
0     -0.240803 -0.379347       7        1   3.687620
2     -0.011072  0.351034       8        1   3.633790
13     0.006288  0.469475       9        1   3.555270
17     0.176864 -0.401561      10        1   3.449816
1      0.352118 -0.270134      11        1   3.417252
21    -0.172771  0.411211      12        1   3.290781
3      0.356077  0.205508      13        1   3.044244
6     -0.304019  0.305564      14        1   2.814363
28     0.160372  0.387079      15        1   2.735205
11    -0.165466  0.229880      16        1   2.356123
9     -0.360903 -0.012638      17        1   2.322244
14    -0.304638  0.141400      18        1   2.290028
29     0.238086  0.053400      19        1   2.125675
20    -0.102333 -0.367872      20        1   2.109823
16     0.313969 -0.075925      21        1   2.084002
7      0.123993  0.202038      22        1   2.061278
27     0.248803  0.285510      23        1   1.974116
23    -0.233325 -0.069003      24        1   1.829723
18     0.196525 -0.244776      25        1   1.826020
5     -0.006892 -0.262765      26        1   1.800061
22    -0.106819 -0.149078      27        1   1.527938
25     0.117082 -0.099057      28        1   1.449412
4     -0.052741  0.048124      29        1   0.584639
15     0.005249 -0.003148      30        1   0.039137, topic_info=        Term        Freq        Total Category  logprob  loglift
26    people  7516.00000  7516.000000  Default   30.000  30.0000
13    matter  8785.00000  8785.000000  Default   29.000  29.0000
43       say  6189.00000  6189.000000  Default   28.000  28.0000
12      life  8289.00000  8289.000000  Default   27.000  27.0000
32        so  3925.00000  3925.000000  Default   26.000  26.0000
..       ...         ...          ...      ...      ...      ...
23  movement     0.00563   635.794404  Topic30   -9.607  -3.7888
25      name     0.00563   402.956147  Topic30   -9.607  -3.3327
26    people     0.00563  7516.175760  Topic30   -9.607  -6.2587
27    police     0.00563  1229.504469  Topic30   -9.607  -4.4482
29    really     0.00563  1463.790341  Topic30   -9.607  -4.6227

[1280 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
342       5  0.992499     abuse
332      20  0.992717       act
44        7  0.998115  actually
6        19  0.995646     again
146       4  0.993714    agenda
...     ...       ...       ...
71       12  0.998035      work
85       12  0.998626     world
207       5  0.998084     wrong
107      14  0.999012      year
192      20  0.991990     young

[331 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 25, 9, 27, 20, 13, 1, 3, 14, 18, 2, 22, 4, 7, 29, 12, 10, 15, 30, 21, 17, 8, 28, 24, 19, 6, 23, 26, 5, 16])